In [2]:
#import other packages for dataset store
import numpy as np
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk

In [63]:
dataset = load_from_disk("dataset_tha")

In [64]:
dataset

DatasetDict({
    test: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story', 'image_path'],
        num_rows: 58
    })
    validation: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story', 'image_path'],
        num_rows: 52
    })
    train: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story', 'image_path'],
        num_rows: 2913
    })
})

In [65]:
import pythainlp
pythainlp.__version__

'3.1.1'

In [101]:
from pythainlp import word_tokenize

#test
text = "เจ้ากระต่ายกระโดดผ่านมา\xa0 มันหวังว่าจะได้กินกล้วย\nสีเทา จึงพูดขึ้นว่า “เจ้ากระต่ายจ๋า\xa0 อีกสามอาทิตย์”\nเจ้ากระต่ายจึงกระโดดผ่านไป"
text = text.replace('\n', '0')

print("default (newmm):")
print(word_tokenize(text)) #list

newcaption = word_tokenize(text)
newcaption = list([x if x is not " " else "0" for x in newcaption])
newcaption = ' '.join(newcaption)
newcaption
#for i in range(len(newcaption)):
    

default (newmm):
['เจ้า', 'กระต่าย', 'กระโดด', 'ผ่าน', 'มา', '\xa0', ' ', 'มัน', 'หวัง', 'ว่า', 'จะ', 'ได้', 'กิน', 'กล้วย', '0', 'สีเทา', ' ', 'จึง', 'พูด', 'ขึ้น', 'ว่า', ' ', '“', 'เจ้า', 'กระต่าย', 'จ๋า', '\xa0', ' ', 'อีก', 'สาม', 'อาทิตย์', '”', '0', 'เจ้า', 'กระต่าย', 'จึง', 'กระโดด', 'ผ่าน', 'ไป']


'เจ้า กระต่าย กระโดด ผ่าน มา \xa0 0 มัน หวัง ว่า จะ ได้ กิน กล้วย 0 สีเทา 0 จึง พูด ขึ้น ว่า 0 “ เจ้า กระต่าย จ๋า \xa0 0 อีก สาม อาทิตย์ ” 0 เจ้า กระต่าย จึง กระโดด ผ่าน ไป'

In [67]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from pythainlp import word_tokenize

def single_token(text):
    #replace \n by 0, because we need to record where \n appeared
    text = text.replace('\n', '0')
    #use thai word tokenizer
    caption2 = word_tokenize(text)
    caption2 = list([x if x is not " " else "0" for x in caption2])
    result = ' '.join(caption2)
    return result

def fetch_caption(batch, num_threads, timeout=None, retries=3):
    fetch_single_image_with_args = partial(single_token)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["caption2"] = list(executor.map(fetch_single_image_with_args, batch["caption"]))
    return batch

num_threads = 20
dataset = dataset.map(fetch_caption, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

In [98]:
dataset['train'][9]

{'image_id': '723f3568-0f5c-43b0-9198-e6afa2c81f5d',
 'image_url': 'https://bloom-vist.s3.amazonaws.com/%EF%BB%BF%E0%B8%9E%E0%B8%A2%E0%B8%B1%E0%B8%8D%E0%B8%8A%E0%B8%99%E0%B8%B0%E0%B8%A7%E0%B8%B8%E0%B9%88%E0%B8%99%E0%B8%A7%E0%B8%B2%E0%B8%A2+%E0%B8%81%E0%B8%B1%E0%B8%9A+%E0%B8%AA%E0%B8%A3%E0%B8%B0%E0%B9%80%E0%B8%89%E0%B8%A2%E0%B8%8A%E0%B8%B2/1.10.jpg',
 'caption': 'พยัญชนะก.ไก่ รีบเข้ามากอดและพูดขอบคุณสระอา ที่เข้ามาช่วยจากนั้นก็ขอโทษสระอา ที่ตนทอดทิ้งเพื่อนรักให้เดินทางมาเพียงลำพัง สระอารับคำขอโทษจากพยัญชนะก.ไก่ แล้วทั้งสองเดินเที่ยวงานกันอย่างสนุกสนาน และพากันกลับอณาจักรสมุดหรรษาอย่างมีควาสุข',
 'story_id': 'f4a9150f-3b0f-491d-8869-4067726f269b',
 'album_id': 'ad14aeaa-c939-4885-8a10-46fbc5b73bc9',
 'license': 'cc-by-nc-nd',
 'original_bloom_language_tag': 'th',
 'index_in_story': 9,
 'image_path': 'images_tha/517b11e0-8bab-427f-a9bb-fbb97777ad04.jpg',
 'caption2': 'พยัญชนะ ก. ไก่ 0 รีบ เข้ามา กอด และ พูด ขอบคุณ สระ อา 0 ที่ เข้ามา ช่วย จากนั้น ก็ ขอโทษ สระ อา 0 ที่ ตน ทอดทิ้ง เพื่อนรัก

In [96]:
#save dataset to local
dataset.save_to_disk("dataset_tha2")

In [93]:
text = "เจ้ากระต่ายกระโดดผ่านมา\xa0 มันหวังว่าจะได้กินกล้วย\nสีเทา จึงพูดขึ้นว่า “เจ้ากระต่ายจ๋า\xa0 อีกสามอาทิตย์”\nเจ้ากระต่ายจึงกระโดดผ่านไป"
text = text.replace(r"!\"#$%&\(\)\*\+.,-/:;=?@\[\\\]^_`{|}~", "")
text = text.replace('\n', '')
#text = text.replace('\xa0', '')
text

'เจ้ากระต่ายกระโดดผ่านมา\xa0 มันหวังว่าจะได้กินกล้วยสีเทา จึงพูดขึ้นว่า “เจ้ากระต่ายจ๋า\xa0 อีกสามอาทิตย์”เจ้ากระต่ายจึงกระโดดผ่านไป'

Post process of predict data

In [40]:
#import other packages for dataset store
import numpy as np
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk

In [68]:
tha_result = pd.read_csv("tha_result_50.csv", encoding='utf-8')

In [69]:
list_of_pred_column = tha_result['predicted'].tolist()

post_pred_column = []
for i in range(len(list_of_pred_column)):
    text = list_of_pred_column[i]
    text = text.replace('\xa0', '')
    text = "".join(text.split())
    text = text.replace('0', ' ')
    for j in text:
        if j == "":
            text.remove(j)
    
    post_pred_column.append(text)
    

In [70]:
del tha_result['predicted']
tha_result.insert(1, 'predicted2', post_pred_column, allow_duplicates=False)
tha_result.to_csv("tha_result3.csv", index=False)


In [66]:
import csv
with open('tha_result3.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write multiple rows
    writer.writerows(tha_result)

In [67]:
see = pd.read_csv("tha_result3.csv", encoding='utf-8')